In [1]:
import cost_model
import os
import json
import sys

from pprint import pprint
from IPython.display import SVG, display, HTML

def show_svg(url1, url2):
    display(HTML("""<table><tr><td><img style='width:100px' src='{}'/></td>
                               <td><img style='width:100px' src='{}'/></td></tr></table>""".format(url1, url2)))

# the folder containing all exports
export_dir = "./saved_designs"

def load_design(design_dir):
    """the directory containing exported designs"""

    design = {}
    # load saved designs and trashed designs
    with open(os.path.join(design_dir, "saved.json")) as f:
        design["saved"] = json.load(f)["saved"]
    with open(os.path.join(design_dir, "trashed.json")) as f:
        design["trashed"] = json.load(f)["trashed"]

    # load paths for saved svgs, relates to saved designs by their canvas ids
    svg_dir = os.path.join(design_dir, "saved_svgs")
    if not os.path.isdir(svg_dir):
        svg_dir = os.path.join(design_dir, "import_these_into_xd")
    design["svg_paths"] = {}
    for fname in os.listdir(svg_dir):
        if fname.endswith(".svg"):
            design_id = fname.split(".")[0].split("-")[-1]
            design["svg_paths"][design_id] = os.path.join(svg_dir, fname)

    design["dirname"] = design_dir
    return design

def analyze_design(design):
    # for each pair of saved designs, display their diversity score as well as svgs
    
    raw_diversity_score = {}
    
    if len(design["saved"]) == 1:
        print("[error] this directory contains only 1 saved desigsn.")
        return
    
    for i in range(len(design["saved"])):
        for j in range(i + 1, len(design["saved"])):
            di, dj = design["saved"][i], design["saved"][j]
            #diversity_score, diff = cost_model.compute_diversity_score(di["elements"], dj["elements"])
            raw_diversity_score[(i, j)] = cost_model.compute_unnormalized_diversity_score(
                di["elements"], dj["elements"])

    final_scores = cost_model.normalize_diversity_scores(raw_diversity_score)
    
    design_pairs = []
    for pair_id in final_scores:
        i,j = pair_id
        score = (final_scores[pair_id]["alt_group_score"] 
                    + final_scores[pair_id]["pos_diff_score"] 
                    + final_scores[pair_id]["rel_dist_diff_score"] 
                    + 2 * final_scores[pair_id]["size_diff_score"])
        design_pair = {
            "score": score,
            "score_detail": final_scores[pair_id],
            "di": design["saved"][i],
            "dj": design["saved"][j],
        }
        
        design_pairs.append(design_pair)

    design_pairs.sort(key=lambda x: x['score'], reverse=True)
    for pair in design_pairs: 
        no_svg_to_display = False
        di = pair['di']
        dj = pair['dj']
        for d in [di, dj]:
            if d["id"] not in design["svg_paths"]:
                #print("  [Error] no svg for id {}".format(d["id"]))
                no_svg_to_display = True

        if no_svg_to_display:
            #print("  " + design["dirname"])
            continue
        score = pair['score']
        score_detail = pair['score_detail']
        print("# diversity score {}".format(score))
        print("# score details")
        pprint(score_detail)

        svg_i = design["svg_paths"][di["id"]]
        svg_j = design["svg_paths"][dj["id"]]
        show_svg(svg_i, svg_j)
        print("")
            
# iterate over all exported deisngs
for dirname in os.listdir(export_dir):
    design_dir = os.path.join(export_dir, dirname)
    if not os.path.isdir(design_dir):
        continue
    designs = load_design(design_dir)
    
    print("==============")
    print("## Analyzing {} ...".format(dirname))
    analyze_design(designs)

## Analyzing exported_from_scout (98) ...
# diversity score 1.5353792462506421
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.6282603104492578,
 'rel_dist_diff_score': 0.4208555099617864,
 'size_diff_score': 0.24313171291979896}


,



# diversity score 1.4510752266040559
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.4540924483411888,
 'rel_dist_diff_score': 0.37559525909339414,
 'size_diff_score': 0.3106937595847364}


,



# diversity score 1.3385973696068607
# score details
{'alt_group_score': 0.07692307692307693,
 'pos_diff_score': 0.44998153473605157,
 'rel_dist_diff_score': 0.3657798689743475,
 'size_diff_score': 0.2229564444866924}


...

,



# diversity score 0.7296028134031944
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.21582188321455975,
 'rel_dist_diff_score': 0.17361659695172632,
 'size_diff_score': 0.17008216661845418}


,



# diversity score 0.694722286008248
# score details
{'alt_group_score': 0.07692307692307693,
 'pos_diff_score': 0.23740616076745905,
 'rel_dist_diff_score': 0.18045644527113844,
 'size_diff_score': 0.09996830152328678}


,
